## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kavieng,PG,-2.5744,150.7967,82.42,81,99,14.67,overcast clouds
1,1,Whitehorse,CA,60.7161,-135.0538,84.02,31,48,5.99,scattered clouds
2,2,Atar,MR,20.5169,-13.0499,99.55,16,98,3.42,overcast clouds
3,3,Rikitea,PF,-23.1203,-134.9692,76.19,73,5,18.23,clear sky
4,4,Lorengau,PG,-2.0226,147.2712,81.73,78,100,6.17,heavy intensity rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                234
City                   234
Country                234
Lat                    234
Lng                    234
Max Temp               234
Humidity               234
Cloudiness             234
Wind Speed             234
Current Description    234
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows
#No null values returned

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
clean_hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
clean_hotel_df["Hotel Name"] = ""
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kavieng,PG,82.42,overcast clouds,-2.5744,150.7967,
1,Whitehorse,CA,84.02,scattered clouds,60.7161,-135.0538,
3,Rikitea,PF,76.19,clear sky,-23.1203,-134.9692,
4,Lorengau,PG,81.73,heavy intensity rain,-2.0226,147.2712,
6,Jamestown,US,78.03,overcast clouds,42.0970,-79.2353,
7,Itarema,BR,77.27,overcast clouds,-2.9248,-39.9167,
12,Atuona,PF,77.40,scattered clouds,-9.8000,-139.0333,
17,Maumere,ID,76.12,broken clouds,-8.6199,122.2111,
19,Cayenne,GF,80.64,clear sky,4.9333,-52.3333,
22,Vaini,TO,77.16,scattered clouds,-21.2000,-175.2000,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in clean_hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        clean_hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kavieng,PG,82.42,overcast clouds,-2.5744,150.7967,Nusa Island Retreat
1,Whitehorse,CA,84.02,scattered clouds,60.7161,-135.0538,The Sternwheeler Hotel & Conference Centre
3,Rikitea,PF,76.19,clear sky,-23.1203,-134.9692,People ThankYou
4,Lorengau,PG,81.73,heavy intensity rain,-2.0226,147.2712,Lorengau Harbourside Hotel
6,Jamestown,US,78.03,overcast clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
7,Itarema,BR,77.27,overcast clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
12,Atuona,PF,77.40,scattered clouds,-9.8000,-139.0333,Villa Enata
17,Maumere,ID,76.12,broken clouds,-8.6199,122.2111,Sylvia Hotel
19,Cayenne,GF,80.64,clear sky,4.9333,-52.3333,Hôtel Le Dronmi
22,Vaini,TO,77.16,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))